In [1]:
from pyquil import get_qc, Program
from pyquil.gates import H, CNOT, MEASURE

In [2]:
# fire once the servers: /src/pyquil/entrypoint.sh 
# Get a quantum virtual machine (simulator)
qvm = get_qc("2q-qvm")

In [3]:
# Build a program
p = Program()
p += H(0)
p += CNOT(0, 1)
ro = p.declare("ro", "BIT", 2)
p += MEASURE(0, ro[0])
p += MEASURE(1, ro[1])
p.wrap_in_numshots_loop(10) # to perform multi-shot execution
print(p)

H 0
CNOT 0 1
DECLARE ro BIT[2]
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [4]:
# Execute the program synchronously
qvm.run(p).readout_data.get("ro")

array([[0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1]])

In [5]:
# Alternately, asynchronous execution is supported
#  run this program on the Quantum Virtual Machine (QVM)
executable = qvm.compile(p)
result = qvm.run(executable)
bitstrings = result.readout_data.get('ro')
print(bitstrings)

[[0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]


In [12]:
print(executable)

DECLARE ro BIT[2]
RZ(pi/2) 0
RX(pi/2) 0
RZ(-pi/2) 1
RX(pi/2) 1
CZ 1 0
RX(-pi/2) 1
RZ(pi/2) 1
MEASURE 1 ro[1]
RZ(-pi/2) 0
MEASURE 0 ro[0]
HALT



In [14]:
# repeat the same execution on simulated HW
device_name = 'Aspen-11'
qvmA11 = get_qc(device_name, as_qvm=True)
executableA11 = qvmA11.compile(p)
print(executableA11)

DECLARE ro BIT[2]
RZ(-pi/2) 0
RX(-pi/2) 0
RZ(pi) 1
XY(pi) 0 1
RZ(-pi/2) 0
RX(-pi/2) 0
RZ(pi/2) 0
XY(pi) 0 1
MEASURE 0 ro[0]
RX(-pi/2) 1
MEASURE 1 ro[1]
HALT



In [20]:
np = qvmA11.compiler.quil_to_native_quil(p, protoquil=True)
print(np.metadata)

AttributeError: 'Program' object has no attribute 'metadata'

In [24]:
list(np)

[<DECLARE ro>,
 <Gate RZ(-pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(pi) 1>,
 <Gate XY(pi) 0 1>,
 <Gate RZ(-pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate XY(pi) 0 1>,
 <Gate RX(-pi/2) 1>,

In [25]:
from pyquil.quil import address_qubits
print(address_qubits(p))

H 0
CNOT 0 1
DECLARE ro BIT[2]
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [1]:
from qiskit_rigetti import RigettiQCSProvider, QuilCircuit